In [43]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from plotly.subplots import make_subplots
from config import fetch_data

## **CHURN PREDICTION**

### Churn Prediction Distribution

In [44]:
churn_prediction_distribution_query = "SELECT * FROM gold.analytics_customer_churn_prediction"
churn_prediction_distribution_df = fetch_data(churn_prediction_distribution_query)

fig = px.histogram(
    churn_prediction_distribution_df,
    x='predicted_churn_risk_pct',
    color='predicted_churn_category',
    title='Predicted Customer Churn Risk Distribution',
    labels={'predicted_churn_risk_pct': 'Predicted Churn Risk (%)'},
    color_discrete_map={
        'Low': '#00CC96',
        'Medium': '#FFA15A',
        'High': '#EF553B',
        'Critical': '#AB63FA'
    },
    nbins=50,
    barmode='overlay',
    opacity=0.75,
    marginal='box',
    hover_data={'predicted_churn_risk_pct': ':.2f'} 
)

# Improve layout
fig.update_layout(
    height=550,
    width=900,
    title={
        'text': "Predicted Customer Churn Risk Distribution",
        'x':0.5,  # Center title
        'xanchor': 'center'
    },
    xaxis_title='Predicted Churn Risk (%)',
    yaxis_title='Number of Customers',
    legend_title_text='Churn Category',
    font=dict(size=12, family='Arial'),
    plot_bgcolor='white',
    paper_bgcolor='white'
)

# Style axes
fig.update_xaxes(showgrid=True, gridwidth=0.5, gridcolor='lightgrey', zeroline=False)
fig.update_yaxes(showgrid=True, gridwidth=0.5, gridcolor='lightgrey', zeroline=False)

# Show chart
fig.show()

In [45]:
churn_prediction_distribution_df

,customer_key,customer_natural_key,customer_segment,tenure_months,historical_churn_risk_pct,predicted_churn_risk_pct,predicted_churn_category,days_since_login,transaction_count_90d,active_account_count,past_due_account_count,unresolved_issues,negative_interactions,clv_at_risk,prediction_date,prediction_window_end
0,dcd3f83c96576c0fd437286a1ff6f1f0,12556,Affluent,85.0,100.0,4850.0,Critical,2471.0,0,0,0,1,1,72936.0,2025-12-27 10:35:04.628469+00:00,2026-03-27
1,64357314e1c294fca2c6419e6b6d59af,13750,Business,56.0,100.0,4850.0,Critical,4267.0,0,0,0,1,0,86466.0,2025-12-27 10:35:04.628469+00:00,2026-03-27
2,b6b90237b3ebd1e462a5d11dbc5c4dae,11266,Business,69.0,100.0,4850.0,Critical,3472.0,0,0,0,1,0,5205.0,2025-12-27 10:35:04.628469+00:00,2026-03-27
3,0120ec28bc96b5b1ce1ba1999ce204b0,18213,Mass Market,30.0,99.0,4820.0,Critical,1281.0,0,0,0,1,0,59249.0,2025-12-27 10:35:04.628469+00:00,2026-03-27
4,a0a052b2340749617466e56d7b8e74ef,16831,Affluent,117.0,99.0,4820.0,Critical,1020.0,0,0,0,1,1,70650.0,2025-12-27 10:35:04.628469+00:00,2026-03-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16107,4e246a381baf2ce038b3b0f82c7d6fb4,6924,Premium,128.0,2.0,510.0,Critical,NaN,1,1,0,0,0,62867.0,2025-12-27 10:35:04.628469+00:00,2026-03-27
16108,ea4617226119a78dda076158cbaef41d,21228,Mass Market,175.0,4.0,500.0,Critical,NaN,2,3,0,0,0,58624.0,2025-12-27 10:35:04.628469+00:00,2026-03-27
16109,b578f2a52a0229873fefc2a4b06377fa,10302,Premium,25.0,1.0,480.0,Critical,NaN,1,2,0,0,0,17711.0,2025-12-27 10:35:04.628469+00:00,2026-03-27
16110,5d4a18500f626b6a8cb05c1b3c4d095a,10168,Mass Market,164.0,0.0,450.0,Critical,NaN,1,3,0,0,0,95789.0,2025-12-27 10:35:04.628469+00:00,2026-03-27


### Churn Risk Funnel 

In [46]:
churn_risk_funnel_query  = """
    SELECT predicted_churn_category, 
            COUNT(*) as customer_count,
            SUM(clv_at_risk) as total_clv_at_risk
    FROM gold.analytics_customer_churn_prediction
    GROUP BY predicted_churn_category
    ORDER BY CASE predicted_churn_category
        WHEN 'Critical' THEN 1
        WHEN 'High' THEN 2
        WHEN 'Medium' THEN 3
        WHEN 'Low' THEN 4
    END
"""

churn_risk_funnel_df = fetch_data(churn_risk_funnel_query)

fig = go.Figure(go.Funnel(
    y=churn_risk_funnel_df['predicted_churn_category'],
    x=churn_risk_funnel_df['customer_count'],
    textinfo="value+percent initial",
    marker={"color": ['#AB63FA', '#EF553B', '#FFA15A', '#00CC96']},
    hovertemplate=(
        '<b>%{y}</b><br>' +
        'Customers: %{x:,}<br>' +
        'CLV at Risk: $%{customdata:,.2f}<extra></extra>'
    ),
    customdata=churn_risk_funnel_df['total_clv_at_risk']
))

# Layout improvements
fig.update_layout(
    title={
        'text': "Customer Churn Risk Funnel",
        'x':0.5,  # Center title
        'xanchor': 'center'
    },
    height=550,
    width=800,
    font=dict(size=12, family='Arial'),
    plot_bgcolor='white',
    paper_bgcolor='white'
)

# Add annotations for total CLV at risk (optional but nice for presentation)
for i, row in churn_risk_funnel_df.iterrows():
    fig.add_annotation(
        x=row['customer_count'],
        y=row['predicted_churn_category'],
        text=f"${row['total_clv_at_risk']:,.0f}",
        showarrow=False,
        xanchor='left',
        yanchor='middle',
        font=dict(size=11, color='black')
    )

fig.show()

In [47]:
churn_risk_funnel_df

,predicted_churn_category,customer_count,total_clv_at_risk
0,Critical,16112,813125255.0


### Churn Feature Importance Scatter

In [48]:
churn_feature_importance_scatter_query = "SELECT * FROM gold.analytics_customer_churn_prediction ORDER BY predicted_churn_risk_pct DESC LIMIT 1000"
churn_feature_importance_scatter_df = fetch_data(churn_feature_importance_scatter_query)

fig = px.scatter(
    churn_feature_importance_scatter_df,
    x='days_since_login',
    y='predicted_churn_risk_pct',
    size='clv_at_risk',
    color='customer_segment',
    hover_data={
        'transaction_count_90d': ':,',
        'active_account_count': ':,',
        'unresolved_issues': ':,',
        'clv_at_risk': ':$,.2f'
    },
    title='Churn Risk Drivers: Days Since Last Login vs Predicted Risk (Top 1000 Customers)',
    labels={
        'days_since_login': 'Days Since Last Login',
        'predicted_churn_risk_pct': 'Predicted Churn Risk (%)',
        'customer_segment': 'Customer Segment'
    },
    color_discrete_sequence=px.colors.qualitative.Plotly,
    size_max=30
)

# Layout improvements
fig.update_layout(
    height=650,
    width=900,
    title={
        'text': 'Churn Risk Drivers: Days Since Last Login vs Predicted Risk (Top 1000 Customers)',
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis=dict(
        title='Days Since Last Login',
        showgrid=True,
        gridcolor='lightgrey'
    ),
    yaxis=dict(
        title='Predicted Churn Risk (%)',
        showgrid=True,
        gridcolor='lightgrey'
    ),
    legend_title_text='Customer Segment',
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(size=12, family='Arial')
)

# Add trendline for visual reference (optional)
fig = px.scatter(
    churn_feature_importance_scatter_df,
    x='days_since_login',
    y='predicted_churn_risk_pct',
    color='customer_segment',
    size='clv_at_risk',
    trendline='ols',  # Adds linear regression trendline
    trendline_scope='overall',
    hover_data={
        'transaction_count_90d': ':,',
        'active_account_count': ':,',
        'unresolved_issues': ':,',
        'clv_at_risk': ':$,.2f'
    },
    size_max=30
)

fig.show()

In [49]:
churn_feature_importance_scatter_df

,customer_key,customer_natural_key,customer_segment,tenure_months,historical_churn_risk_pct,predicted_churn_risk_pct,predicted_churn_category,days_since_login,transaction_count_90d,active_account_count,past_due_account_count,unresolved_issues,negative_interactions,clv_at_risk,prediction_date,prediction_window_end
0,dcd3f83c96576c0fd437286a1ff6f1f0,12556,Affluent,85.0,100.0,4850.0,Critical,2471.0,0,0,0,1,1,72936.0,2025-12-27 10:35:04.628469+00:00,2026-03-27
1,64357314e1c294fca2c6419e6b6d59af,13750,Business,56.0,100.0,4850.0,Critical,4267.0,0,0,0,1,0,86466.0,2025-12-27 10:35:04.628469+00:00,2026-03-27
2,b6b90237b3ebd1e462a5d11dbc5c4dae,11266,Business,69.0,100.0,4850.0,Critical,3472.0,0,0,0,1,0,5205.0,2025-12-27 10:35:04.628469+00:00,2026-03-27
3,7e6ff0205749bc6025b51155e26f6ced,4817,Business,155.0,99.0,4820.0,Critical,3011.0,0,0,0,2,1,71713.0,2025-12-27 10:35:04.628469+00:00,2026-03-27
4,0120ec28bc96b5b1ce1ba1999ce204b0,18213,Mass Market,30.0,99.0,4820.0,Critical,1281.0,0,0,0,1,0,59249.0,2025-12-27 10:35:04.628469+00:00,2026-03-27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,9f93557d309f655ff06f109a08dcf7c4,5107,Business,131.0,96.0,3980.0,Critical,3095.0,0,2,0,0,0,92260.0,2025-12-27 10:35:04.628469+00:00,2026-03-27
996,2979158b9c1e72755a8961f03e3580f5,19047,Premium,117.0,96.0,3980.0,Critical,2216.0,0,4,0,0,0,71814.0,2025-12-27 10:35:04.628469+00:00,2026-03-27
997,3f53d7190148675e3cd472fc826828c5,2355,Premium,80.0,96.0,3980.0,Critical,4968.0,0,1,0,0,0,8032.0,2025-12-27 10:35:04.628469+00:00,2026-03-27
998,10907813b97e249163587e6246612e21,2245,Premium,105.0,96.0,3980.0,Critical,4964.0,0,2,0,0,0,23581.0,2025-12-27 10:35:04.628469+00:00,2026-03-27


## **TRANSACTION FORECASTING**

### Transaction Forecast with Bands

In [50]:
transaction_forecast_with_bands_query = "SELECT * FROM gold.analytics_transaction_volume_forecast ORDER BY date_actual DESC"
transaction_forecast_with_bands_df = fetch_data(transaction_forecast_with_bands_query)

fig = go.Figure()

# Actual values
fig.add_trace(go.Scatter(
    x=transaction_forecast_with_bands_df['date_actual'],
    y=transaction_forecast_with_bands_df['actual_count'],
    mode='lines',
    name='Actual',
    line=dict(color='#636EFA', width=2),
    hovertemplate='Date: %{x}<br>Actual: %{y:,}<extra></extra>'
))

# Forecast
fig.add_trace(go.Scatter(
    x=transaction_forecast_with_bands_df['date_actual'],
    y=transaction_forecast_with_bands_df['forecasted_count'],
    mode='lines',
    name='Forecast',
    line=dict(color='#00CC96', width=2, dash='dash'),
    hovertemplate='Date: %{x}<br>Forecast: %{y:,}<extra></extra>'
))

# Upper confidence band
fig.add_trace(go.Scatter(
    x=transaction_forecast_with_bands_df['date_actual'],
    y=transaction_forecast_with_bands_df['forecast_upper_bound'],
    mode='lines',
    line=dict(width=0),
    showlegend=False,
    hoverinfo='skip'
))

# Lower confidence band with fill
fig.add_trace(go.Scatter(
    x=transaction_forecast_with_bands_df['date_actual'],
    y=transaction_forecast_with_bands_df['forecast_lower_bound'],
    mode='lines',
    line=dict(width=0),
    fillcolor='rgba(0, 204, 150, 0.2)',
    fill='tonexty',
    name='95% Confidence',
    hovertemplate='Date: %{x}<br>Lower: %{y:,}<extra></extra>'
))

fig.update_layout(
    title="Transaction Volume Forecast with 95% Prediction Interval (90 Days)",
    xaxis_title="Date",
    yaxis_title="Transaction Count",
    height=500,
    hovermode='x unified'
)

fig.show()

In [51]:
transaction_forecast_with_bands_df

,date_actual,day_of_week,day_name,is_weekend,month,actual_count,actual_volume,forecasted_count,forecasted_volume,forecast_lower_bound,forecast_upper_bound,forecast_error_pct,generated_at
0,2025-12-24,3.0,Wednesday,False,12.0,27,45327.53,30.0,50363.92,21.0,39.0,11.11,2025-12-27 10:35:13.314308+00:00
1,2025-12-23,2.0,Tuesday,False,12.0,30,49391.20,31.0,51037.57,22.0,40.0,3.33,2025-12-27 10:35:13.314308+00:00
2,2025-12-22,1.0,Monday,False,12.0,29,56576.23,31.0,60478.04,22.0,40.0,6.90,2025-12-27 10:35:13.314308+00:00
3,2025-12-21,0.0,Sunday,True,12.0,35,52356.80,31.0,46373.17,22.0,40.0,11.43,2025-12-27 10:35:13.314308+00:00
4,2025-12-20,6.0,Saturday,True,12.0,31,54766.15,31.0,54766.15,22.0,40.0,0.00,2025-12-27 10:35:13.314308+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
83,2025-10-02,4.0,Thursday,False,10.0,33,61746.27,30.0,56132.97,17.0,43.0,9.09,2025-12-27 10:35:13.314308+00:00
84,2025-10-01,3.0,Wednesday,False,10.0,21,36993.34,30.0,52847.63,17.0,43.0,42.86,2025-12-27 10:35:13.314308+00:00
85,2025-09-30,2.0,Tuesday,False,9.0,27,58307.11,31.0,66945.20,18.0,44.0,14.81,2025-12-27 10:35:13.314308+00:00
86,2025-09-29,1.0,Monday,False,9.0,25,35555.20,31.0,44088.45,19.0,43.0,24.00,2025-12-27 10:35:13.314308+00:00


### Forecast Accuracy Metrics

In [52]:
forecast_accuracy_metrics_query = """
    SELECT 
    AVG(forecast_error_pct) as avg_error_pct,
    STDDEV(forecast_error_pct) as stddev_error_pct,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY forecast_error_pct) as median_error_pct
    FROM gold.analytics_transaction_volume_forecast
    WHERE actual_count IS NOT NULL
"""

forecast_accuracy_metrics_df = fetch_data(forecast_accuracy_metrics_query)

fig = make_subplots(
    rows=1, cols=3,
    specs=[[{"type": "indicator"}, {
        "type": "indicator"}, {"type": "indicator"}]],
    subplot_titles=("Avg Forecast Error",
                    "Median Error", "Error Volatility")
)

fig.add_trace(
    go.Indicator(
        mode="gauge+number",
        value=forecast_accuracy_metrics_df['avg_error_pct'].iloc[0],
        title={'text': "%"},
        gauge={
            'axis': {'range': [0, 20]},
            'bar': {'color': "darkblue"},
            'steps': [
                {'range': [0, 5], 'color': "lightgreen"},
                {'range': [5, 10], 'color': "yellow"},
                {'range': [10, 20], 'color': "red"}
            ]
        }
    ),
    row=1, col=1
)

fig.add_trace(
    go.Indicator(
        mode="number",
        value=forecast_accuracy_metrics_df['median_error_pct'].iloc[0],
        number={'suffix': "%"},
        title={'text': "Median"}
    ),
    row=1, col=2
)

fig.add_trace(
    go.Indicator(
        mode="number",
        value=forecast_accuracy_metrics_df['stddev_error_pct'].iloc[0],
        number={'suffix': "%"},
        title={'text': "StdDev"}
    ),
    row=1, col=3
)

fig.update_layout(
    title_text="Forecast Accuracy Dashboard",
    height=400
)

fig.show()

In [53]:
forecast_accuracy_metrics_df

,avg_error_pct,stddev_error_pct,median_error_pct
0,13.57875,9.897383,11.94


## **LOAN DEFAULT PREDICTION**

### Loan Default Risk Matrix

In [54]:
loan_default_risk_matrix_query = "SELECT * FROM gold.analytics_loan_default_prediction WHERE total_payments >= 5"
loan_default_risk_matrix_df = fetch_data(loan_default_risk_matrix_query)

fig = px.scatter(
    loan_default_risk_matrix_df,
    x='default_probability_pct',
    y='avg_outstanding_balance',
    size='avg_outstanding_balance',
    color='default_risk_category',
    hover_data={
        'default_probability_pct': ':.1f%',
        'avg_outstanding_balance': ':$,.2f',
        'late_payment_count': ':,',
        'missed_payment_count': ':,'
    },
    title='Loan Default Risk Matrix: Probability vs Expected Loss',
    labels={
        'default_probability_pct': 'Default Probability (%)',
        'avg_outstanding_balance': 'Expected Loss ($)'
    },
    color_discrete_map={
        'Low': '#00CC96',
        'Medium': '#FFA15A',
        'High': '#EF553B',
        'Critical': '#AB63FA'
    }
)

# Add quadrant lines
fig.add_hline(y=loan_default_risk_matrix_df['avg_outstanding_balance'].median(),
                line_dash="dash", line_color="gray", opacity=0.5)
fig.add_vline(x=loan_default_risk_matrix_df['default_probability_pct'].median(),
                line_dash="dash", line_color="gray", opacity=0.5)

# Add quadrant labels
fig.add_annotation(x=75, y=loan_default_risk_matrix_df['avg_outstanding_balance'].max()*0.9, text="High Risk<br>High Impact",
                    showarrow=False, font=dict(size=12, color="red"))
fig.add_annotation(x=25, y=loan_default_risk_matrix_df['avg_outstanding_balance'].max()*0.9, text="Low Risk<br>High Impact",
                    showarrow=False, font=dict(size=12, color="orange"))

fig.update_layout(height=600)
fig.show()

In [55]:
loan_default_risk_matrix_df

,account_key,account_natural_key,customer_key,credit_score,credit_score_band,income_bracket,total_payments,late_payment_count,missed_payment_count,avg_days_late,max_days_late,total_late_fees,recent_late_payments,avg_outstanding_balance,max_outstanding_balance,interest_rate_pct,rate_category,default_probability_pct,default_risk_category
0,9c7fe1658be50fa6f26e326f9c1ff9ec,37374,44c1fda479266012f488a7cc8c45f1a9,552,Poor,High,19,3,1,0.842105,7,116.146536,3,19523.567774,26203.193175,5.5,Medium Rate,100.0,Critical
1,33097b67fa51e0115312196ee8231b5b,17043,8a7129b8f3edd95b7d969dfc2c8e9d9d,478,Poor,Very High,50,6,4,0.780000,13,248.239699,6,91498.344644,123558.613806,5.5,Medium Rate,100.0,Critical
2,3314dfc68a97404aeecc86a670e50766,32242,e8ad3f3f04296aa9be9de71a674e3769,631,Fair,Very High,37,6,1,1.459459,14,203.024850,6,268504.547940,350898.253254,4.5,Low Rate,100.0,Critical
3,33169e6c57113f6de6b49fb5d8d32838,15390,9a0ee0a9e7a42d2d69b8f86b3a0756b1,547,Poor,Very High,39,10,5,2.230769,15,429.790644,10,258847.119852,352323.796647,4.5,Low Rate,100.0,Critical
4,331ffa63ab5f2d214d68a6cbbddb3095,48685,3f1d00695d5aac23f5c556054b88be47,761,Very Good,Very High,59,7,6,0.983051,15,271.037533,7,130076.448460,172185.498971,5.5,Medium Rate,100.0,Critical
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18554,ef6fe50336005eee1e8e9093bfe576ae,14312,ea81a3d20bf98ef2c9bef9dc24ec777a,748,Very Good,Very High,12,0,0,0.000000,0,0.000000,0,284161.107295,366742.777412,8.0,Medium Rate,5.0,Low
18555,f95ec3de395b4bce25b39ef6138da871,3610,3210ddbeaa16948a702b6049b8d9a202,787,Very Good,Very High,6,0,0,0.000000,0,0.000000,0,341124.615087,416443.499368,5.5,Medium Rate,5.0,Low
18556,fa2246fa0fdf0d3e270c86767b77ba1b,11882,7f278ad602c7f47aa76d1bfc90f20263,800,Excellent,Low,10,0,0,0.000000,0,0.000000,0,121465.063614,150593.635087,4.5,Low Rate,5.0,Low
18557,e25ca237a194723ab3c86e793660ef21,46719,077ab52e20c26fc6bde433e9d0c13bf1,764,Very Good,Medium,6,0,0,0.000000,0,0.000000,0,142300.975391,175103.407540,8.0,Medium Rate,5.0,Low


### Load Default Distribution

In [56]:
loan_default_distribution_query = "SELECT * FROM gold.analytics_loan_default_prediction"
loan_default_distribution_df = fetch_data(loan_default_distribution_query)

# Color mapping
risk_colors = {'Low': '#00CC96', 'Medium': '#FFA15A', 'High': '#EF553B', 'Critical': '#AB63FA'}

# Create violin plot
fig = go.Figure()

for risk_cat in ['Low', 'Medium', 'High', 'Critical']:
    risk_data = loan_default_distribution_df[loan_default_distribution_df['default_risk_category'] == risk_cat]
    
    fig.add_trace(go.Violin(
        y=risk_data['default_probability_pct'],
        name=risk_cat,
        box_visible=True,
        meanline_visible=True,
        line_color=risk_colors[risk_cat],
        fillcolor=risk_colors[risk_cat],
        opacity=0.6,
        points='all',  # Show individual points
        hovertemplate='<b>%{fullData.name}</b><br>Probability: %{y:.1f}%<extra></extra>'
    ))

# Layout improvements
fig.update_layout(
    title={
        'text': "Default Probability Distribution by Risk Category",
        'x': 0.5,
        'xanchor': 'center'
    },
    yaxis_title="Default Probability (%)",
    xaxis_title="Risk Category",
    height=550,
    width=850,
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(size=12, family='Arial'),
    violingap=0.2,  # spacing between violins
    violinmode='group',  # group violins side by side
    legend_title_text='Risk Category',
    showlegend=True
)

# Style axes
fig.update_yaxes(showgrid=True, gridcolor='lightgrey', zeroline=False)
fig.update_xaxes(showgrid=False)

fig.show()

In [57]:
loan_default_distribution_df

,account_key,account_natural_key,customer_key,credit_score,credit_score_band,income_bracket,total_payments,late_payment_count,missed_payment_count,avg_days_late,max_days_late,total_late_fees,recent_late_payments,avg_outstanding_balance,max_outstanding_balance,interest_rate_pct,rate_category,default_probability_pct,default_risk_category
0,9c7fe1658be50fa6f26e326f9c1ff9ec,37374,44c1fda479266012f488a7cc8c45f1a9,552,Poor,High,19,3,1,0.842105,7,116.146536,3,19523.567774,26203.193175,5.5,Medium Rate,100.0,Critical
1,33097b67fa51e0115312196ee8231b5b,17043,8a7129b8f3edd95b7d969dfc2c8e9d9d,478,Poor,Very High,50,6,4,0.780000,13,248.239699,6,91498.344644,123558.613806,5.5,Medium Rate,100.0,Critical
2,3314dfc68a97404aeecc86a670e50766,32242,e8ad3f3f04296aa9be9de71a674e3769,631,Fair,Very High,37,6,1,1.459459,14,203.024850,6,268504.547940,350898.253254,4.5,Low Rate,100.0,Critical
3,33169e6c57113f6de6b49fb5d8d32838,15390,9a0ee0a9e7a42d2d69b8f86b3a0756b1,547,Poor,Very High,39,10,5,2.230769,15,429.790644,10,258847.119852,352323.796647,4.5,Low Rate,100.0,Critical
4,331ffa63ab5f2d214d68a6cbbddb3095,48685,3f1d00695d5aac23f5c556054b88be47,761,Very Good,Very High,59,7,6,0.983051,15,271.037533,7,130076.448460,172185.498971,5.5,Medium Rate,100.0,Critical
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18749,fa2246fa0fdf0d3e270c86767b77ba1b,11882,7f278ad602c7f47aa76d1bfc90f20263,800,Excellent,Low,10,0,0,0.000000,0,0.000000,0,121465.063614,150593.635087,4.5,Low Rate,5.0,Low
18750,fbe6ce6a3f92aa796ef05b887d97a891,29110,4572101f572c37f203cd8690fe6e4eeb,783,Very Good,High,4,0,0,0.000000,0,0.000000,0,63531.543457,88788.641812,8.0,Medium Rate,5.0,Low
18751,e25ca237a194723ab3c86e793660ef21,46719,077ab52e20c26fc6bde433e9d0c13bf1,764,Very Good,Medium,6,0,0,0.000000,0,0.000000,0,142300.975391,175103.407540,8.0,Medium Rate,5.0,Low
18752,ef76495350b28fa7e59ddcb8afb52b47,57854,06f46a725a4bdc070e07b09bb7d05dce,781,Very Good,Very High,4,0,0,0.000000,0,0.000000,0,252390.078862,346026.663897,5.5,Medium Rate,5.0,Low


## **CLV PREDICTION**

### CLV Prediction Scatter

In [58]:
clv_prediction_scatter_query = "SELECT * FROM gold.analytics_customer_lifetime_value_prediction ORDER BY predicted_clv_36m DESC"
clv_prediction_scatter_df = fetch_data(clv_prediction_scatter_query)

clv_prediction_scatter_df['size_for_plot'] = clv_prediction_scatter_df['expected_value_increase_36m'].abs()

# Create scatter plot
fig = px.scatter(
    clv_prediction_scatter_df,
    x='current_clv',
    y='predicted_clv_36m',
    color='predicted_value_tier',
    size='size_for_plot',
    hover_data={
        'current_clv': ':$,.2f',
        'predicted_clv_36m': ':$,.2f',
        'growth_rate_12m_pct': ':.1f%',
        'monthly_avg_volume': ':$,.2f',
        'account_count': ':,',
        'expected_value_increase_36m': ':$,.2f'
    },
    title='Customer Lifetime Value: Current vs 36-Month Prediction (Top 1000)',
    labels={
        'current_clv': 'Current CLV ($)',
        'predicted_clv_36m': 'Predicted CLV 36m ($)'
    },
    size_max=30
)

# Add diagonal line (no growth)
max_val = max(clv_prediction_scatter_df['current_clv'].max(), clv_prediction_scatter_df['predicted_clv_36m'].max())
fig.add_trace(go.Scatter(
    x=[0, max_val],
    y=[0, max_val],
    mode='lines',
    line=dict(dash='dash', color='gray', width=2),
    name='No Growth Line',
    hoverinfo='skip'
))

# Layout improvements
fig.update_layout(
    height=600,
    width=900,
    xaxis=dict(title='Current CLV ($)', showgrid=True, gridcolor='lightgrey'),
    yaxis=dict(title='Predicted CLV 36m ($)', showgrid=True, gridcolor='lightgrey'),
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(size=12, family='Arial')
)

fig.show()

In [59]:
clv_prediction_scatter_df

,customer_key,customer_natural_key,customer_segment,tenure_months,current_clv,annual_income,age,monthly_avg_transactions,monthly_avg_volume,total_balance,account_count,segment_upgrades,predicted_clv_12m,predicted_clv_36m,expected_value_increase_12m,expected_value_increase_36m,growth_rate_12m_pct,predicted_value_tier,size_for_plot
0,d2849eee3432ef804bfccac1a9cb24f0,6465,Mass Market,6.0,82377,445050,27,17.000000,25790.260,-21765.90,5,0,98233.50,148624.31,15856.50,66247.31,19.25,High Value,66247.31
1,c9513b89892fb5a2667053143887285a,17840,Premium,6.0,95505,20630,51,15.000000,27218.550,-701352.56,3,0,105182.60,147641.93,9677.60,52136.93,10.13,High Value,52136.93
2,d11875b53233adc482d07142c4b80ffa,20467,Business,169.0,82264,341432,64,22.000000,46837.040,-321429.32,4,1,96891.04,144613.60,14627.04,62349.60,17.78,High Value,62349.60
3,905056c1ac1dad141560467e0a99e1cf,1246,Affluent,4.0,93864,362854,61,10.500000,15434.220,266118.09,3,0,106145.71,140491.19,12281.71,46627.19,13.08,High Value,46627.19
4,5d7ee5401d8717dfb57e50a32f60eaf2,18959,Mass Market,2.0,98089,67046,52,11.000000,16110.490,20792.78,2,0,108203.22,139927.25,10114.22,41838.25,10.31,High Value,41838.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16107,3e1fb9297735057d44263e674078a7fd,10218,Affluent,43.0,1009,289463,30,0.000000,0.000,-665160.56,2,1,-4902.61,-13400.01,-5911.61,-14409.01,-585.89,Standard Value,14409.01
16108,1abf005ea3f957c5cb79941ab7707c11,7380,Affluent,6.0,2144,264971,19,2.200000,2880.730,-1003698.40,4,0,-5684.55,-14249.02,-7828.55,-16393.02,-365.14,Standard Value,16393.02
16109,83d5a9ee8867f73d6f2c485d89296d1d,14453,Premium,5.0,12736,463026,56,0.600000,816.694,-1238345.64,3,0,1202.56,-15084.57,-11533.44,-27820.57,-90.56,Standard Value,27820.57
16110,cd9508fdaa5c1390e9cc329001cf1459,6665,Affluent,117.0,3956,441923,19,1.444444,1678.130,-924598.07,3,0,-4325.85,-15662.44,-8281.85,-19618.44,-209.35,Standard Value,19618.44


### CLV Growth Segments

In [60]:
clv_growth_segments_query = """
    SELECT customer_segment,
        AVG(current_clv) as avg_current_clv,
        AVG(predicted_clv_36m) as avg_predicted_clv,
        AVG(growth_rate_12m_pct) as avg_growth_rate,
        COUNT(*) as customer_count
    FROM gold.analytics_customer_lifetime_value_prediction
    GROUP BY customer_segment
    ORDER BY avg_predicted_clv DESC
"""

clv_growth_segments_df = fetch_data(clv_growth_segments_query)

fig = go.Figure()

# Current CLV
fig.add_trace(go.Bar(
    name='Current CLV',
    x=clv_growth_segments_df['customer_segment'],
    y=clv_growth_segments_df['avg_current_clv'],
    marker_color='#636EFA',
    hovertemplate='<b>%{x}</b><br>Current CLV: $%{y:,.2f}<extra></extra>'
))

# Predicted CLV
fig.add_trace(go.Bar(
    name='Predicted CLV (36m)',
    x=clv_growth_segments_df['customer_segment'],
    y=clv_growth_segments_df['avg_predicted_clv'],
    marker_color='#00CC96',
    hovertemplate='<b>%{x}</b><br>Predicted CLV: $%{y:,.2f}<br>Growth: %{customdata:.1f}%<extra></extra>',
    customdata=clv_growth_segments_df['avg_growth_rate']
))

# Layout improvements
fig.update_layout(
    title={
        'text': "Average CLV Growth Prediction by Customer Segment",
        'x': 0.5,  # Center title
        'xanchor': 'center'
    },
    xaxis_title="Customer Segment",
    yaxis_title="CLV ($)",
    barmode='group',
    height=550,
    width=900,
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(size=12, family='Arial'),
    legend_title_text='CLV Type'
)

# Add data labels on top of bars (optional but nice for presentations)
for i, row in clv_growth_segments_df.iterrows():
    fig.add_annotation(x=row['customer_segment'], y=row['avg_current_clv'],
                       text=f"${row['avg_current_clv']:,.0f}",
                       showarrow=False, yanchor='bottom', font=dict(size=11, color='#636EFA'))
    fig.add_annotation(x=row['customer_segment'], y=row['avg_predicted_clv'],
                       text=f"${row['avg_predicted_clv']:,.0f}",
                       showarrow=False, yanchor='bottom', font=dict(size=11, color='#00CC96'))

fig.show()

In [61]:
clv_growth_segments_df

,customer_segment,avg_current_clv,avg_predicted_clv,avg_growth_rate,customer_count
0,Business,50908.540317,54134.310487,3.783386,3981
1,Affluent,50527.470617,53601.999477,2.552073,4033
2,Mass Market,50269.852686,53276.002798,3.444892,4039
3,Premium,50170.271495,52954.011298,2.787689,4059


## **FRAUD RISK FORECASTING**

### Fraud Risk Forecast Heatmap

In [62]:
fraud_risk_forecast_heatmap_query = """
    SELECT merchant_category, channel, 
        AVG(predicted_fraud_risk_pct) as avg_risk,
        SUM(expected_fraud_count_30d) as total_expected_fraud
    FROM gold.analytics_fraud_risk_forecast
    GROUP BY merchant_category, channel
    ORDER BY avg_risk DESC
    LIMIT 50
"""
fraud_risk_forecast_heatmap_df = fetch_data(fraud_risk_forecast_heatmap_query)

pivot = fraud_risk_forecast_heatmap_df.pivot_table(
    values='avg_risk',
    index='merchant_category',
    columns='channel',
    fill_value=0
)

fig = go.Figure(data=go.Heatmap(
    z=pivot.values,
    x=pivot.columns,
    y=pivot.index,
    colorscale='Reds',
    text=pivot.values,
    texttemplate='%{text:.2f}%',
    textfont={"size": 9},
    hovertemplate='<b>%{y}</b> - <b>%{x}</b><br>Predicted Risk: %{z:.2f}%<extra></extra>',
    colorbar=dict(title="Fraud Risk %")
))

fig.update_layout(
    title="Predicted Fraud Risk Heatmap: Category × Channel",
    xaxis_title="Channel",
    yaxis_title="Merchant Category",
    height=700
)

In [63]:
fraud_risk_forecast_heatmap_df

,merchant_category,channel,avg_risk,total_expected_fraud
0,Gas Station,Branch,0.4175,1.0
1,Retail,POS,0.3775,1.0
2,Services,ATM,0.3650,2.0
3,Utilities,ATM,0.2900,0.0
4,Services,Mobile,0.2725,0.0
5,Healthcare,Mobile,0.2600,0.0
6,Utilities,POS,0.2600,1.0
7,Utilities,Branch,0.2575,1.0
8,Travel,Mobile,0.2575,0.0
9,Travel,Online,0.2525,1.0
